In [11]:
import pandas as pd
import numpy as np
import base64
import re

from sklearn.metrics.pairwise import cosine_similarity

import random

In [12]:
def str_to_vec(str):
    return np.frombuffer(base64.b85decode(str), dtype = np.float32)
def vec_to_str(vec):
    return base64.b85encode(vec.tobytes()).decode()

In [14]:
f = open('txt/importance_vectors.txt', 'r')
importance_vectors = list(map(str_to_vec ,eval(f.read())))

In [15]:
f = open('txt/category_vectors.txt', 'r')
category_vectors = list(map(str_to_vec ,eval(f.read())))

In [17]:
def calc_user_iv(importance_ratings):
    sum_user_iv = importance_vectors[0] * importance_ratings[0]
    for i in range(1, len(importance_ratings)):
        sum_user_iv = sum_user_iv + importance_vectors[i] * importance_ratings[i]
    user_iv = sum_user_iv/sum(importance_ratings)
    return user_iv

In [18]:
def calc_user_cv(category_ratings):
    sum_user_cv = category_vectors[0] * category_ratings[0]
    for i in range(1, len(category_ratings)):
        sum_user_cv = sum_user_cv + category_vectors[i] * category_ratings[i]
    user_cv = sum_user_cv/sum(category_ratings)
    return user_cv

In [19]:
def calc_test_user_vector(importance_ratings, category_ratings):
    user_iv = calc_user_iv(importance_ratings)
    user_cv = calc_user_cv(category_ratings)
    user_vector = 0.2*user_iv + 0.8*user_cv
    return user_vector

In [21]:
place_df = pd.read_csv('data/places_ML.csv')

In [23]:
"""

3 -> 0
5 -> 2
5 -> 2
1 -> -2

(0a + 2b + 2c - 2d) / 2
(1a + 2b + 2c - 1d) / 4

14  2
4


"""

'\n\n3 -> 0\n5 -> 2\n5 -> 2\n1 -> -2\n\n(0a + 2b + 2c - 2d) / 2\n(1a + 2b + 2c - 1d) / 4\n\n14  2\n4\n\n\n'

In [24]:
def rating_place(user_vector, count_rating_places, place_id, rating):
    place_vector = str_to_vec(place_df.loc[place_df['naver_id']==place_id, 'place_vector'].values[0])
    result_user_vector is None
    if count_rating_places == 0:
        result_user_vector = (user_vector*0.7 + place_vector*(rating/5)*0.3)
    else:
        result_user_vector = (user_vector*count_rating_places + place_vector*(rating/5)) / (count_rating_places + 1)
    
    return result_user_vector

In [35]:
# result_user_vector = calc_place_rating(user_vector, 6, 1336760258, 5)

In [42]:
def read_place_vectors(place_df_vector):
    document_embedding_list = []        
    i = 0

    # 각 문서에 대해서
    for v_tostring in place_df_vector:
        document_embedding_list.append(str_to_vec(v_tostring))
    
    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

In [45]:
def pick_places(user_vector):
    place_ids = []
    
    document_embedding_list = read_place_vectors(place_df['place_vector'])
    cosine_similarities = cosine_similarity([user_vector], document_embedding_list)

    # 입력된 장소와 리뷰(document embedding)가 유사한 장소 10개 선정.
    sim_scores = list(enumerate(cosine_similarities[0]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[0:10]
    
    # 가장 유사한 장소 10개의 인덱스
    place_indices = [i[0] for i in sim_scores]

    # 10개의 장소 중 5개 랜덤 선택
    random_indices = random.sample(place_indices, 5)

    for i in random_indices:
        place_ids.append(int(place_df.loc[i, 'naver_id']))

    return place_ids

In [47]:
def test(importance_ratings, category_ratings):

    return pick_places(calc_test_user_vector(importance_ratings, category_ratings))

In [49]:
# user_vector = calc_test_user_vector([1, 2, 3, 0, 1], [2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2,1,1,1,1,1])